In [1]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import time


def connect_robot(ip="192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip)  # IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

In [2]:
from capture_image import capture_image
import sys
import os
import warnings
import numpy as np

# add parent folder to sys.path
sys.path.insert(0, os.path.abspath(".."))

# ignore DeprecationWarning messages
warnings.filterwarnings("ignore", category=DeprecationWarning)

# ignore RuntimeWarning messages
warnings.filterwarnings("ignore", category=RuntimeWarning)

from volume_estimation.src.models_1_no_vol.predict_full_pipeline import predict_no_vol
from segmentation_and_depth.src.models.predict_model import predict as predict_seg_depth
from volume_estimation.src.models_2_input_vol.predict_full_pipeline import (
    predict_with_vol,
)

In [3]:
rtde_c, rtde_r, rtde_io_set = connect_robot()
print("Activating Gripper")
gripper = RobotiqGripper(rtde_c)
gripper.activate()  # returns to previous position after activation
gripper.set_force(0)  # from 0 to 100 %
gripper.set_speed(10)  # from 0 to 100 %
print("Gripper activated")

Activating Gripper
Gripper activated


In [4]:
j_start = [-3.141514603291647,
 -0.39998288572344975,
 -1.7599776983261108,
 2.149993820781372,
 -1.5708015600787562,
 -1.5499151388751429]

j_up = [-1.2547920385943812,
 -1.1487035018256684,
 -0.9265894889831543,
 3.4698049265095214,
 -2.3617973963366907,
 -2.868312184010641]

j_bottles_pic = [-0.3581450621234339,
 -1.9314056835570277,
 -1.2249127626419067,
 1.648374004954956,
 -1.5516436735736292,
 -1.853001896535055]

j_miscroscope = [-3.523916546498434,
 -1.4186616402915497,
 -2.1420812606811523,
 3.26844231664624,
 -1.840297047291891,
 -3.248759094868795]

j_flasks_1 = [-3.149257485066549,
 -1.2185992461493989,
 -2.3591017723083496,
 2.720782919520996,
 -1.07640248933901,
 -2.7709713617907923]

j_bottle_holder = [-3.169065777455465,
 -1.727936407128805,
 -2.395404815673828,
 2.8663622575947265,
 -0.6110208670245569,
 -1.927342716847555]

j_flask_holders_trash = [-3.504859511052267,
 -1.2505887907794495,
 -2.19888973236084,
 2.0256845194050292,
 -0.8002708593951624,
 -1.601645294819967]

j_flask_storage_1 = [-4.287753526364462,
 -1.8319250545897425,
 -1.2737982273101807,
 3.1778737741657714,
 0.3764202296733856,
 -3.236988369618551]

j_flask_storage_2 = [-4.287816349660055,
 -1.8942696056761683,
 -1.6332104206085205,
 3.597304506892822,
 0.3639531135559082,
 -3.235102717076437]

j_flask_storage_3 = [-4.288263742123739,
 -2.134270807305807,
 -1.8128883838653564,
 4.010412855739258,
 0.36713337898254395,
 -3.228081766759054]


In [19]:
def check_microscope():
    flask_present = False
    path = capture_image()
    print("Path: ", path)
    predict_no_vol(path, save_segmentation=True, save_depth=False, predict_volume=False, no_GPU=True)
    segmentation_map_npy = np.load(path[:-4] + "_VesselMask.npy")
    # value count of segmentation map
    unique, counts = np.unique(segmentation_map_npy, return_counts=True)
    print("Value count of segmentation map: ", dict(zip(unique, counts)))

    # if counts only contains 0, then vessel is closed:
    if len(counts) == 1:
        print("No flask present.")
        return flask_present
    if counts[1] > 10000:
        print("Flask present. Please remove flask from microscope holder.")
        flask_present = True
        return flask_present
    else:
        print("No flask present.")
        return flask_present
    
def check_flasks_1():
    flask_present = False
    path = capture_image()
    print("Path: ", path)
    predict_no_vol(path, save_segmentation=True, save_depth=False, predict_volume=False, no_GPU=True)
    segmentation_map_npy = np.load(path[:-4] + "_VesselMask.npy")
    # value count of segmentation map
    unique, counts = np.unique(segmentation_map_npy, return_counts=True)
    print("Value count of segmentation map: ", dict(zip(unique, counts)))

    # if counts only contains 0, then vessel is closed:
    if len(counts) == 1:
        print("No flask present.")
        return flask_present
    if counts[1] > 5000:
        print("Flask present. Please remove flask from flask holder.")
        flask_present = True
        return flask_present
    else:
        print("No flask present.")
        return flask_present
    
def check_bottle_holder():
    flask_present = False
    path = capture_image()
    print("Path: ", path)
    predict_no_vol(path, save_segmentation=True, save_depth=False, predict_volume=False, no_GPU=True)
    segmentation_map_npy = np.load(path[:-4] + "_VesselMask.npy")
    # value count of segmentation map
    unique, counts = np.unique(segmentation_map_npy, return_counts=True)
    print("Value count of segmentation map: ", dict(zip(unique, counts)))

    # if counts only contains 0, then vessel is closed:
    if len(counts) == 1:
        print("No bottle present.")
        return flask_present
    if counts[1] > 5000:
        print("Bottle present. Please remove bottle from bottle holder.")
        flask_present = True
        return flask_present
    else:
        print("No bottle present.")
        return flask_present
    
def check_flask_holder_and_trash():
    flask_present = False
    trash_present = True
    path = capture_image()
    print("Path: ", path)
    predict_no_vol(path, save_segmentation=True, save_depth=False, predict_volume=False, no_GPU=True)
    segmentation_map_npy = np.load(path[:-4] + "_VesselMask.npy")
    # value count of segmentation map
    unique, counts = np.unique(segmentation_map_npy, return_counts=True)
    print("Value count of segmentation map: ", dict(zip(unique, counts)))

    if counts[1] < 5000:
        print("No trash present. Please add trash bottle.")
        trash_present = False
        return flask_present, trash_present
    if counts[1] < 28000:
        print("Everything okay.")
        return flask_present, trash_present
    else:
        print("Flask present. Please remove flask from flask holder.")
        flask_present = True
        return flask_present, trash_present
    
        
def check_flask_storage():
    flask_1_present = False
    flask_2_present = False
    path = capture_image()
    print("Path: ", path)
    predict_no_vol(path, save_segmentation=True, save_depth=False, predict_volume=False, no_GPU=True)
    segmentation_map_npy = np.load(path[:-4] + "_VesselMask.npy")
    # value count of segmentation map
    unique, counts = np.unique(segmentation_map_npy, return_counts=True)
    print("Value count of segmentation map: ", dict(zip(unique, counts)))

    if counts[1] < 3000:
        return flask_1_present, flask_2_present
    if counts[1] < 5000:
        return flask_present, trash_present
    else:
        print("Flask present. Please remove flask from flask holder.")
        flask_present = True
        return flask_present, trash_present

In [4]:
init_q = rtde_r.getActualQ()
init_q

[-1.2547271887408655,
 -1.1486415725997468,
 -0.9266389608383179,
 3.469817801112793,
 -2.3618019262896937,
 -2.868333164845602]

In [4]:
rtde_c.moveJ(j_start, 0.2, 0.2)

True

In [6]:
rtde_c.moveJ(j_up, 0.2, 0.2)

True

In [7]:
rtde_c.moveJ(j_bottles_pic, 0.2, 0.2)

True

In [8]:
rtde_c.moveJ(j_up, 0.2, 0.2)

True

In [ ]:
rtde_c.moveJ(j_start, 0.2, 0.2)

In [16]:
rtde_c.moveJ(j_miscroscope, 0.2, 0.2)

True

In [21]:
check_microscope()

Image saved
Path:  captured_images/20230710_161309/image.png
Value count of segmentation map:  {0: 203499, 1: 103701}
Flask present. Please remove flask from microscope holder.


True

In [23]:
rtde_c.moveJ(j_flasks_1, 0.2, 0.2)

True

In [30]:
check_flasks_1()

Image saved
Path:  captured_images/20230710_161810/image.png
Value count of segmentation map:  {0: 258952, 1: 48248}
Flask present. Please remove flask from flask holder.


True

In [6]:
rtde_c.moveJ(j_bottle_holder, 0.2, 0.2)

True

In [9]:
check_bottle_holder()

Image saved
Path:  captured_images/20230710_162131/image.png
Value count of segmentation map:  {0: 208699, 1: 98501}
Bottle present. Please remove bottle from bottle holder.


True

In [10]:
rtde_c.moveJ(j_flask_holders_trash, 0.2, 0.2)

True

In [17]:
check_flask_holder_and_trash()

Image saved
Path:  captured_images/20230710_162905/image.png
Value count of segmentation map:  {0: 260436, 1: 46764}
Flask present. Please remove flask from flask holder.


(True, True)

In [18]:
rtde_c.moveJ(j_flask_storage_1, 0.2, 0.2)

True

In [10]:
rtde_c.moveJ(j_flask_storage_2, 0.2, 0.2)

True

In [11]:
rtde_c.moveJ(j_flask_storage_3, 0.2, 0.2)

True

In [12]:
rtde_c.moveJ(j_start, 0.2, 0.2)

True

In [22]:
def check_setup(rtde_c):
    rtde_c.moveJ(j_start, 0.7, 0.7)
    rtde_c.moveJ(j_miscroscope, 0.7, 0.7)
    path_microscope = capture_image()

    rtde_c.moveJ(j_flasks_1, 0.7, 0.7)
    path_flasks_1 = capture_image()

    rtde_c.moveJ(j_bottle_holder, 0.7, 0.7)
    path_bottle_holder = capture_image()

    rtde_c.moveJ(j_flask_holders_trash, 0.7, 0.7)
    path_flask_holders_trash = capture_image()

    rtde_c.moveJ(j_flask_storage_1,0.7, 0.7)
    path_flask_storage_1 = capture_image()

    rtde_c.moveJ(j_flask_storage_2, 0.7, 0.7)
    path_flask_storage_2 = capture_image()

    rtde_c.moveJ(j_flask_storage_3, 0.7, 0.7)
    path_flask_storage_3 = capture_image()

    rtde_c.moveJ(j_start, 0.7, 0.7)

    return path_microscope, path_flasks_1, path_bottle_holder, path_flask_holders_trash, path_flask_storage_1, path_flask_storage_2, path_flask_storage_3



In [23]:
path_microscope, path_flasks_1, path_bottle_holder, path_flask_holders_trash, path_flask_storage_1, path_flask_storage_2, path_flask_storage_3 = check_setup(rtde_c)

Image saved
Image saved
Image saved
Image saved
Image saved
Image saved
Image saved
